In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
)
model = AutoModelForCausalLM.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  pad_token_id=tokenizer.eos_token_id,
  torch_dtype='auto', low_cpu_mem_usage=False
).to(device='cuda', non_blocking=True)
model.eval()
print()

In [2]:
def generate_sentences(model, tokenizer, text, length):
    tokenized_text = tokenizer(text, return_tensors='pt').to('cuda')
    generated_ids = model.generate(
        tokenized_text.input_ids,
        do_sample=True, #샘플링 전략 사용
        max_length=(len(tokenized_text.input_ids[0]) + length), # 최대 디코딩 길이
        top_k=10, # 확률 순위 밖인 토큰은 샘플링에서 제외
        top_p=0.6, # 누적 확률 이내의 후보집합에서만 생성
        no_repeat_ngram_size = 4,
        num_return_sequences = 5 # 한 번에 출력할 다음 문장 선택지의 개수
    ).detach().to('cpu')
    
    generated_texts = [tokenizer.decode(id, skip_special_tokens=True) for id in generated_ids]
    return generated_texts

In [3]:
sample_text = input("시작 텍스트를 입력하세요> ");

In [4]:
last_sentence = sample_text + ''
max_len = 700
new_sentence_length = 64

while len(sample_text) <= max_len:
    new_sentences = generate_sentences(model, tokenizer, sample_text, new_sentence_length)
    new_sentences = [sent.split(last_sentence)[-1].replace('\n', ' ') for sent in new_sentences]
    print("[기존 텍스트의 마지막 문장]\n" + last_sentence)
    print("\n다음 문장을 선택하세요.\n문장을 새로 입력받고 싶은 경우 번호를 제외한 아무 입력이나 입력하세요.\n")
    print("현재 글자수:", str(len(sample_text)))
    for i, sent in enumerate(new_sentences):
        print(f"[{i+1}] " + sent.strip())
   
    try:
        selection = input(">")
        if selection == "exit":
            break
        selection = int(selection)
        if selection < 1 or selection > len(new_sentences):
            continue
        last_sentence = new_sentences[selection-1]
    except:
        continue
        
    sample_text = sample_text + last_sentence

[기존 텍스트의 마지막 문장]
GPT-3의 텍스트 생성 능력을 테스트하기 위한 데모 입니다. 글의 시작이 되는 첫 문장을 입력하면 다음에 오기 자연스러운 문장들을 제시해 줍니다. 

다음 문장을 선택하세요.
문장을 새로 입력받고 싶은 경우 번호를 제외한 아무 입력이나 입력하세요.

현재 글자수: 83
[1] 이 테스트를 통해서 텍스트를 생성하는 능력을 평가해 볼 수 있습니다.  테스트 결과입니다.  텍스트 생성 능력은 매우 높습니다.  하지만, 글의 첫 문장을 생성하는 부분에서는 다소 부족한 모습을 보여주고 있습니다.   이 테스트를 통해 텍스트를 생성
[2] 이 데모는 글을 쓰는 사람이 직접 문장을 생성하고 그 문장을 다른 사람이 읽고 평가하는 방식으로 진행됩니다.  테스트는 총 3단계로 진행됩니다. 1. 문장 생성 2. 문장 평가 3. 문장 평가 결과에 따라 점수를 매겨주면 됩니다.  문장
[3] :) ​ ​ ​ ​ 테스트에 사용된 문장들은 아래와 같습니다. ​ ​ ​ "I am a doctor," "I am from Seoul," "My name is Kim." "I'm a doctor." "My father is a doctor. I'm a nurse.
[4] :) ​ ​ ​ ​ 1. 시작하기 전에 ​ 1. 이 글을 쓰는 이유는 무엇인가요? 2. 글을 쓰는데 가장 중요하게 생각한 것은 무엇인가요? 3. 글을 쓸 때 가장 어려웠던 점은 무엇인가요. ​ ​ ​ 2. 글 쓰기 전
[5] 이 테스트를 통해 글의 시작이 어떻게 되는지를 알 수 있습니다. 테스트는 3개의 문장을 보여주고, 그 중에서 첫 문장을 고르는 것입니다. 이 테스트는 텍스트 생성 능력의 향상을 보여줍니다.  테스트를 하기 위해서는 먼저 테스트할 문장을 작성해야 합니다
[기존 텍스트의 마지막 문장]
GPT-3의 텍스트 생성 능력을 테스트하기 위한 데모 입니다. 글의 시작이 되는 첫 문장을 입력하면 다음에 오기 자연스러운 문장들을 제시해 줍니다. 

다음 문장을 선택하세요.
문장을 새로 입력받고

In [5]:
print(sample_text)

GPT-3의 텍스트 생성 능력을 테스트하기 위한 데모 입니다. 글의 시작이 되는 첫 문장을 입력하면 다음에 오기 자연스러운 문장들을 제시해 줍니다. :) 테스트를 위해, 텍스트를 생성하는 것은 아니고, 그냥 글을 읽는 것만으로도 충분히 테스트가 가능합니다. :) 이 테스트는 텍스트를 생성하기 위한 것이 아니라, 텍스트를 읽을 때의 텍스트 생성능력을 테스트 하기 위한 것입니다. :) 텍스트를 읽으면서 자연스럽게 글을 읽고, 글을 읽으면서 자연스레 글을 읽을 수 있는 능력을 테스트 하는 것이죠. :) 저는 테스트를 위해 이 글을 읽었습니다. :) (테스트를 위해 이글을 읽는 분들은 없으시겠죠?) 테스트를 위해 글을 읽은 후, 자연스럽게 읽은 글을 읽어 보았습니다. :(이 테스트를 위해 읽은 글은, "[T]he"라는 단어를 포함하고 있는 문장이었습니다. 이 테스트를 위해 저는 이 글을 "[T:]"라는 단어가 포함된 문장을 읽었을 때, 자연스럽고 자연스럽게 이 글을 이해할 수 있는지를 테스트 했습니다. :-) 이 테스트는, 텍스트를 만들어 내는 것이 아니기 때문에, 테스트를 위해 문장을 읽고 자연스럽고, 자연스럽지 않은 문장을 고르는 것만 하면 됩니다. :) 그럼, 이 테스트의 결과를 한번 볼까요? :) 이 글은, 제가 읽은 글이었습니다 :) 이글을 테스트 하면서, 저는 자연스럽고 부자연스러운 문장을 찾았습니다. 이 글을 테스트 하고 난 후, 저는 이 테스트가 제가 읽는 글을 테스트 한 것이라는 것을 알았습니다. 그리고, 이 테스트를 통해 제가 읽고 있는 글을 읽는데에 문제가 없다는 것을 알게 되었습니다. (이
